In [20]:
import requests
import json
from bs4 import BeautifulSoup

In [47]:
d = ""
with open("allDataTBCP1-P4IDS.json") as file:
    d = json.loads(file.read())

In [22]:
class Item:
    def __init__(self, id, npcID, npcName, kill, quest, questID, recipe, recipeID, dropChance, zone):
        self.ID = id
        self.Obtain = {}
        self.Obtain["NpcID"] = npcID
        self.Obtain["NpcName"] = npcName
        self.Obtain["Kill"] = kill
        self.Obtain["Quest"] = quest
        self.Obtain["QuestID"] = questID
        self.Obtain["Recipe"] = recipe
        self.Obtain["RecipeID"] = recipeID
        self.Obtain["DropChance"] = dropChance
        self.Obtain["Zone"] = zone
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)

In [23]:
def get_region(content, start, end = None):
    idx_s = content.index(start)
    idx_e = len(content)
    if end:
        idx_e = content.index(end, idx_s)
    return content[idx_s + len(start): idx_e]

In [24]:

def GetZone(zoneID):
        zUrl = ZoneUrl + str(zoneID)
        pageZone = requests.get(zUrl)
        soupZone = BeautifulSoup(pageZone.content, 'html.parser')
        zoneNameH1 = soupZone.find('h1', class_='heading-size-1')
        zoneName = zoneNameH1.text
        return zoneName

In [25]:
def GetCategory(categoryID, category2ID):
        cUrl = QuestCaregoryUrl + str(category2ID) + "." + str(categoryID)
        pageCat = requests.get(cUrl)
        soupCat = BeautifulSoup(pageCat.content, 'html.parser')
        CatNameH1 = soupCat.find('h1', class_='heading-size-1')
        zoneName = CatNameH1.text.replace(" Quests", "")
        return zoneName

In [26]:
def GetName(scripts, scriptStart, dataStart, dataEnd):
    data_container = None
    for child in scripts:
        content = ''.join(child.contents).replace('\n   ', '').strip()
        if content.startswith(scriptStart):
            data_container = content
            break
    r0 = get_region(data_container, dataStart)
    r1 = get_region(r0, ' ', dataEnd)
    return r1[9:]

In [27]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [45]:
URL = 'https://tbc.wowhead.com/item='
QuestCaregoryUrl = 'https://tbc.wowhead.com/quests='
ZoneUrl = 'https://tbc.wowhead.com/zone='

SCRIPT_START = "var tabsRelated = new Tabs({parent: WH.ge('jkbfksdbl4'), trackable: 'Item'});"
DATA_START = 'data:'

def GetItem(itemID):
    if (itemID == 0):
        return Item(itemID, 0, "", False, False, 0, False, 0, 0, "").toJson()
    data_container = None
    o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, "")
    url = URL + str(itemID)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    main = soup.find(id='main-contents')
    scripts = main.findAll('script')
    for child in scripts:
        content = ''.join(child.contents).replace('\n   ', '').strip()
        if content.startswith(SCRIPT_START):
            data_container = content
            if (len(content) < 80):
                print("Scraping Item", itemID, "----", "")
                return Item(itemID, 0, "", False, False, 0, False, 0, 0, "").toJson()
            break
    r0 = get_region(data_container, DATA_START)
    r1 = get_region(r0, ' ', ',\n});')
    acqName = GetName(scripts, SCRIPT_START, "name:", ",").lower()
    print("Scraping item", itemID, "----", acqName)
    
    data = json.loads(r1)
    usefulData = data[0]
    zoneName = "Unknown"
    if (acqName == "droppedby"): #Dropped by kill
        dropChance = 0
        for i in data:
            if ('percentOverride' in i):
                dropChance = truncate(i['percentOverride'], 1)
            if ('count' in i):
                if (i['count'] >= 0 and i['outof'] > 0):
                    if (truncate(int(i['count'])/int(i['outof']) * 100, 1) > dropChance):
                        usefulData = i
                        dropChance = truncate(int(usefulData['count'])/int(usefulData['outof']) * 100, 1)
        # Get location
        if ('location' in usefulData):
            zoneName = GetZone(usefulData['location'][0])
        o = Item(itemID, usefulData['id'], usefulData['name'], True, False, 0, False, 0, str(dropChance), zoneName)

    elif (acqName == "soldby"): #Sold by npc
        dropChance = 100
        o = Item(itemID, usefulData['id'], usefulData['name'] + " (Purchase)", True, False, 0, False, 0, str(dropChance), zoneName)

    elif (acqName == "containedin"): #Contained in cache
        dropChance = 0
        print(usefulData)
        for i in data:
            if ('count' in i):
                if (i['count'] >= 0 and i['outof'] > 0):
                    if (truncate(int(i['count'])/int(i['outof']) * 100, 1) > dropChance):
                        usefulData = i
                        dropChance = truncate(int(usefulData['count'])/int(usefulData['outof']) * 100, 1)
        if ('location' in usefulData):
            zoneName = GetZone(usefulData['location'][0])
        o = Item(itemID, usefulData['id'], usefulData['name'] + " (Container)", True, False, 0, False, 0, str(dropChance), zoneName)

    elif (acqName == "rewardfrom"): #Reward from quest
        if ('category' in usefulData and 'category2' in usefulData):
            zoneName = GetCategory(usefulData['category'], usefulData['category2'])
        o = Item(itemID, 0, "", False, True, usefulData['id'], False, 0, 0, zoneName)

    elif (acqName == "createdby"): #Crafted
        o = Item(itemID, 0, "", False, False, 0, True, usefulData['id'], 0, "")

    elif (acqName == "samemodelas_stc"): #Unknown (Usually pvp)
        o = Item(itemID, 0, "", False, False, 0, False, 0, 0, "")
    return o.toJson()


In [46]:
print(GetItem(34352))

Scraping item 34352 ---- droppedby
{"ID": 34352, "Obtain": {"NpcID": 25038, "NpcName": "Felmyst", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": "27.2", "Zone": "Sunwell Plateau"}}


In [49]:
nd = {}
for k, v in d.items():
    print("Scraping class", k) # Class
    nd[k] = {}
    for k2, v2 in v.items():
        for k3, v3 in v2.items():
            ns = k2 + " - " + k3
            print("Scraping Set", ns) # Phase
            nd[k][ns] = {}
            nd[k][ns]["Head"] = GetItem(d[k][k2][k3]["Head"]["itemID"])
            nd[k][ns]["Neck"] = GetItem(d[k][k2][k3]["Neck"]["itemID"])
            nd[k][ns]["Shoulder"] = GetItem(d[k][k2][k3]["Shoulder"]["itemID"])
            nd[k][ns]["Back"] = GetItem(d[k][k2][k3]["Cloak"]["itemID"])
            nd[k][ns]["Chest"] = GetItem(d[k][k2][k3]["Chest"]["itemID"])
            nd[k][ns]["Shirt"] = GetItem(0)
            nd[k][ns]["Tabard"] = GetItem(0)
            nd[k][ns]["Wrists"] = GetItem(d[k][k2][k3]["Wrist"]["itemID"])
            nd[k][ns]["Hands"] = GetItem(d[k][k2][k3]["Gloves"]["itemID"])
            nd[k][ns]["Waist"] = GetItem(d[k][k2][k3]["Waist"]["itemID"])
            nd[k][ns]["Legs"] = GetItem(d[k][k2][k3]["Legs"]["itemID"])
            nd[k][ns]["Feet"] = GetItem(d[k][k2][k3]["Boots"]["itemID"])
            nd[k][ns]["Finger"] = GetItem(d[k][k2][k3]["Ring1"]["itemID"])
            nd[k][ns]["RFinger"] = GetItem(d[k][k2][k3]["Ring2"]["itemID"])
            nd[k][ns]["Trinket"] = GetItem(d[k][k2][k3]["Trinket1"]["itemID"])
            nd[k][ns]["RTrinket"] = GetItem(d[k][k2][k3]["Trinket2"]["itemID"])
            nd[k][ns]["MainHand"] = GetItem(d[k][k2][k3]["MainHand"]["itemID"])
            nd[k][ns]["SecondaryHand"] = GetItem(d[k][k2][k3]["OffHand"]["itemID"])
            nd[k][ns]["Relic"] = GetItem(d[k][k2][k3]["Ranged"]["itemID"])
print("---Done with scrape---")

 32331 ---- droppedby
Scraping item 32592 ---- droppedby
Scraping item 30870 ---- droppedby
Scraping item 31008 ---- samemodelas_stc
Scraping item 30888 ---- droppedby
Scraping item 30916 ---- droppedby
Scraping item 32239 ---- droppedby
Scraping item 32527 ---- droppedby
Scraping item 32247 ---- droppedby
Scraping item 32483 ---- droppedby
Scraping item 31856 ---- rewardfrom
Scraping item 32237 ---- droppedby
Scraping item 30872 ---- droppedby
Scraping Item 28248 ---- 
Scraping Set Elemental - P4
Scraping item 34340 ---- droppedby
Scraping item 34359 ---- createdby
Scraping item 34390 ---- samemodelas_stc
Scraping item 34242 ---- droppedby
Scraping item 34364 ---- createdby
Scraping item 34437 ---- samemodelas_stc
Scraping item 34350 ---- droppedby
Scraping item 34542 ---- samemodelas_stc
Scraping item 34186 ---- droppedby
Scraping item 34566 ---- samemodelas_stc
Scraping item 34230 ---- droppedby
Scraping item 32527 ---- droppedby
Scraping item 33829 ---- droppedby
Scraping item 3185

In [40]:
#URL = 'https://classic.wowhead.com/item='
#QuestCaregoryUrl = 'https://classic.wowhead.com/quests=0.'
#ZoneUrl = 'https://classic.wowhead.com/zone='
#SCRIPT_START = 'WH.Gatherer.addData('
#DATA_START = '$.extend('
#
#
#
#def GetItem(itemID):
#    if (itemID == 0):
#        return Item(itemID, 0, "", False, False, 0, False, 0, 0, "").toJson()
#
#    print("Scraping Item", itemID)
#    url = URL+str(itemID)
#    page = requests.get(url)
#    soup = BeautifulSoup(page.content, 'html.parser')
#
#    main = soup.find(id='main-contents')
#    scripts = main.findAll('script')
#    data_container = None
#    o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, "")
#
#    #Find script tag
#    for child in scripts:
#        content = ''.join(child.contents).replace('\n    ', '').strip()
#        if content.startswith(SCRIPT_START):
#            data_container = content
#            break
#    r0 = get_region(data_container, DATA_START)
#    r1 = get_region(r0, ', ', ');')
#    data = json.loads(r1)
#    if ('sourcemore' in data):
#        usefulData = data['sourcemore'][0]
#        if ('t' in usefulData):
#            if usefulData['t'] == 1: # Kill
#                ZoneName = "Unknown Location"
#                if ('z' in usefulData):
#                    zUrl = ZoneUrl + str(usefulData['z'])
#                    pageZone = requests.get(zUrl)
#                    soupZone = BeautifulSoup(pageZone.content, 'html.parser')
#                    ZoneNameH1 = soupZone.find('h1', class_='heading-size-1')
#                    ZoneName = ZoneNameH1.text
#                dropDiv = soup.find('div', class_="whtt-dropchance")
#                dropChance = 0
#                if (dropDiv is not None):
#                    dropChance = dropDiv.text.replace("Drop Chance: ", "").replace("%", "")
#                o = Item(itemID, usefulData['ti'], usefulData['n'], True, False, 0, False, 0, dropChance, ZoneName)
#            elif usefulData['t'] == 5: # Quest
#                cUrl = QuestCaregoryUrl + str(usefulData['c'])
#                pageCat = requests.get(cUrl)
#                soupCat = BeautifulSoup(pageCat.content, 'html.parser')
#                CatNameH1 = soupCat.find('h1', class_='heading-size-1')
#                ZoneName = CatNameH1.text.replace(" Quests", "")
#                o = Item(itemID, 0, "", False, True, usefulData['ti'], False, 0, 0, ZoneName)
#            elif usefulData['t'] == 6: # Recipe
#                o = Item(itemID, 0, "", False, False, 0, True, usefulData['ti'], 0, "")
#        else:
#            if ('z' in usefulData):
#                zUrl = ZoneUrl + str(usefulData['z'])
#                pageZone = requests.get(zUrl)
#                soupZone = BeautifulSoup(pageZone.content, 'html.parser')
#                ZoneNameH1 = soupZone.find('h1', class_='heading-size-1')
#                ZoneName = ZoneNameH1.text
#                print("---------No type")
#                o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, ZoneName)
#    else:
#        print("---------No Sourcemore")
#    return o.toJson()


In [50]:
print(json.dumps(nd).replace('"{', '{').replace('}"', '}').replace('\\', ''))

47, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Legs": {"ID": 34381, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Feet": {"ID": 34568, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Finger": {"ID": 34213, "Obtain": {"NpcID": 25741, "NpcName": "M'uru", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": "31.5", "Zone": "Sunwell Plateau"}}, "RFinger": {"ID": 29296, "Obtain": {"NpcID": 0, "NpcName": "Unknown acquisition method", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Trinket": {"ID": 34473, "Obtain": {"NpcID": 24560, "NpcName": "Priestess Delrissa", "Kill": true, "Quest": false, "Que

In [51]:
newFile = open("BiSDataTBCP1-4", "w")
newFile.write(json.dumps(nd).replace('"{', '{').replace('}"', '}').replace('\\', ''))
newFile.close()